This is some quik-n-dirty code to get some rough numbers.
I'm not proud of it, but it does the job.

I neglect the shifts in the diagonals of the capactance matrix for
now, as they're all very minor.



## Boilerplate



Setup code and module imports



In [1]:
import numpy as np
import pint
from math import *
from cmc_formulas import *
import cmc_formulas.typical_values as T
from cmc_formulas.typical_values import *

## Typical Values



Here, we print the typical values that i've taken from
(Blais, Alexandre and Huang, Ren-Shou and Wallraff, Andreas and Girvin, S. M. and Schoelkopf, R. J., 2004) and (Gao, Yvonne Y. and Rol, M. Adriaan and Touzard, Steven and Wang, Chen, 2021).



### Resonator



In [1]:
T.CPW

namespace(Z_r=50 <Unit('ohm')>,
          v_0=130000000.0 <Unit('meter / second')>,
          d=1 <Unit('centimeter')>,
          l=153846.15384615381 <Unit('femtofarad / meter')>,
          c=384.6153846153846 <Unit('nanohenry / meter')>,
          C_r=1538.461538461538 <Unit('femtofarad')>,
          L_r=3.846153846153846 <Unit('nanohenry')>,
          ω_0=40.84070449666731 <Unit('gigahertz')>)

In [1]:
T.CPW.ω_0 = Q_(1.2, "gigahertz") * 2 * np.pi

### Qubit



In [1]:
T.Transmon

namespace(ratio=50,
          E_C=200 <Unit('dirac_constant * megahertz * pi')>,
          E_J=10000 <Unit('dirac_constant * megahertz * pi')>,
          C_Σ=193.70229324659127 <Unit('femtofarad')>,
          ω_q=11.938052083641214 <Unit('gigahertz')>,
          α=0.6283185307179586 <Unit('gigahertz')>)

In [1]:
T.Transmon.ω_q / (2*np.pi)

1.9 gigahertz

In [1]:
(T.Transmon.E_J/T.Transmon.E_C).to(ureg.dimensionless)

50.0 dimensionless

In [1]:
formulas.ω_p(T.Transmon.E_C, T.Transmon.E_J) / (2*np.pi)

2.0 gigahertz

## Design Goals



-   we want to be able to intialize a qubit in one corner, run the
    dynamics and read out
-   to lessen the costraints we detune the qubit from the others: the
    diabatic limit tells us of fast we should detune
-   the safe detuning shoudln't be too small compared to the qubit
    frequency to reduce suszeptibility to noise
-   all lamb shifts should be smaller than the safe detuning



## Our Qubit



Let's make a standard transmon with a ratio of $E_J/E_C= 50$ and a
frequency of $\SI{5}{\giga\hertz}$.



In [1]:
transmon = T.TransmonQubit.from_angular_frequency_and_transmon_ratio(5 * ureg.gigahertz * 2 * np.pi, 50)
print(transmon.overview())

ω:      31.41592653589793 gigahertz
ω/2π:   5.0 gigahertz
E_J:    82.67349088394192 dirac_constant * gigahertz
E_C:    1.6534698176788385 dirac_constant * gigahertz
f_J:    13.157894736842106 gigahertz * planck_constant
f_C:    0.2631578947368421 gigahertz * planck_constant
ratio:  50.0 dimensionless

## Bus CPW



I'm just eyeballing this for now, but the bus should be below the
qubit so that we have some leeway with tuning the qubit interaction
and the readout strength.

A detuning of a gigahertz is generally a good idea



In [1]:
bus = T.CoplanarWaveguide.from_ω(ω = 2 * pi * 4 * ureg.gigahertz)
bus.f

4.000000000000001 gigahertz

In [1]:
bus.length

1.6250000000000002 centimeter

This produces a detuning



In [1]:
T.Δ(transmon, bus) / (2*np.pi)

0.9999999999999994 gigahertz

The coherence time of Transmons in on the order of
$\SI{80}{\micro\second}$ (Blais, Alexandre and Grimsmo, Arne L. and Girvin, S. M. and Wallraff, Andreas, 2021), so that the oscillation
half-time $T_{1/2}$ time in [The Model for the CMC Project](../../knowledge/the_model_for_the_cmc_project.md) should be
shorter than that. The detuning speed limit says that
$T_{1/2}\gg\SI{.007}{\micro\second}$.

This value is not set in stone, but i think it is a good starting
point: short enough so that the "safe" detuning is not too small, long
enough to comfortably allow for the detuning of a qubit.



In [1]:
T_12 = .1 * ureg.microsecond
J = np.pi / T_12 * ureg.hbar
(J / (ureg.hbar)).to(ureg.gigahertz)

0.031415926535897934 gigahertz

This is how far we have to detune to completely stop the dynamics.



In [1]:
safe_detune = (J / (ureg.hbar)).to(ureg.megahertz) * 100
safe_detune / (2 * np.pi)

499.99999999999994 megahertz

Detuning one qubit that far completely stops the dynamics.

This basically constraints the coupling strength: if it was any larger
we would have to detune farther&#x2026; (500 MHz seem fine).

Detuning this far can be achieved in (Blais, Alexandre and Grimsmo, Arne L. and Girvin, S. M. and Wallraff, Andreas, 2021)



In [1]:
τ_Δ = (safe_detune  / T.detune_speed).to(ureg.nanosecond)
τ_Δ

0.49999999999999994 nanosecond

This is extremely far below $T_{1/2}$.



In [1]:
(τ_Δ / T_12).to(ureg.dimensionless)

0.004999999999999999 dimensionless

The detuning speed is set a little optimistic anyways.

With this we can calculate the coupling capacitance between Transmon
and bus.



In [1]:
cc = formulas.C_c(T_12, transmon.C_Σ, transmon.E_J, bus.ω)
cc

3.729804378666111 femtofarad

The corresponding oscillator/qubit coupling strength $g$ is



In [1]:
g_bus = T.g(transmon, bus, cc); g_bus / (2*np.pi)

0.04999999999999995 gigahertz

The ratio $g/Δ$ is



In [1]:
T.g(transmon, bus, cc)/T.Δ(transmon, bus)

0.049999999999999975 dimensionless

In [1]:
critical_photon_number(transmon, bus, cc_ro)

15.402137452740726 dimensionless

Let's look at the lamb-shift.



In [1]:
T.lamb_shift(transmon, bus, cc) / transmon.ω

0.0004999999999999991 dimensionless

So the exchange term is negligible.



In [1]:
(T.exchange_coupling(transmon, bus, cc) / J).to(ureg.dimensionless)

0.06578947368421045 dimensionless

The decoupling ratio $Δ/Δ'$ is



In [1]:
T.Δ(transmon, bus) / (T.Δ(transmon, bus) + 1 * ureg.gigahertz * 2 * np.pi)

0.49999999999999983 dimensionless

which is not much&#x2026; but we can't go any better&#x2026;

It is important to keep $C_c\ll C_Σ,C_B$ and $g\ll \abs{Δ}\lesssim ω_q +
ω_B$. Otherwise, everything variale.
Also, the anharmonicity is much smaller than $Δ$. Basically
everything in the following should be small ($\sim .01$).



In [1]:
print(T.dispersive_metrics(transmon, bus, cc))

{'Anharmonicity, α/Δ': 0.26315789473684226, 'Coupling Strength, g/Δ': 0.049999999999999975, 'RWA counter-rotating': 0.11111111111111104, 'Coupling/Transmon Cap': 0.05067195909862064, 'Coupling/CPW Cap': 0.0014919217514664446}

As we can see, there is a lot of play in most things, but the
anharmonicity may be reduced. This can be done either by reducing the
qubit frequency while keeping the detuning constant or decreasing the
transmon ratio $E_J/E_C$. \*Note that the CPW internal capacitance is
only a typical value here. I need more details on the actual design to
be more precise.



## Readout



As we detune and then measure, we are not constrained by $T_{1/2}$
here. I chose the values to be similar to (Jeffrey, Evan and Sank, Daniel and Mutus, J. Y. and White, T. C. and Kelly, J. and Barends, R. and Chen, Y. and Chen, Z. and Chiaro, B. and Dunsworth, A. and Megrant, A. and O’Malley, P. J. J. and Neill, C. and Roushan, P. and Vainsencher, A. and Wenner, J. and Cleland, A. N. and Martinis, John M., 2014) (similar
$g/Δ$).



In [1]:
readout_transmon = T.TransmonQubit.from_angular_frequency_and_transmon_ratio(transmon.ω + safe_detune, 50)

readout = T.CoplanarWaveguide.from_ω(ω = 2 * pi * 1.26 * ureg.gigahertz + readout_transmon.ω)
readout.f, readout_transmon.f

| 6.76 | <Unit | (gigahertz) | > | 5.499999999999999 | <Unit | (gigahertz) | > |

This produces a detuning



In [1]:
T.Δ(readout_transmon, readout) / (2*np.pi)

-1.260000000000001 gigahertz

The corresponding oscillator/qubit coupling strength $g$ is



In [1]:
cc_ro = 4 * ureg.femtofarad
T.g(readout_transmon, readout, cc_ro) / (2*np.pi)

0.09968351212375548 gigahertz

The ratio $g/Δ$ is



In [1]:
T.g(readout_transmon, readout, cc_ro)/T.Δ(readout_transmon, readout)

-0.079113898510917 dimensionless

In [1]:
critical_photon_number(readout_transmon, readout, cc_ro)

19.801150541789227 dimensionless

Seems ok.

The dispersive shift $χ$ is



In [1]:
(T.dispersive_shift(readout_transmon, readout, cc_ro) / (2 * np.pi)).to(ureg.megahertz)

-1.473333285921808 megahertz

which is not *extremely* large. (Walter, T. and Kurpiers, P. and Gasparinetti, S. and Magnard, P. and Potočnik, A. and Salathé, Y. and Pechal, M. and Mondal, M. and Oppliger, M. and Eichler, C. and Wallraff, A., 2017) had 7 megahertz.

The lamb shift of the qubit (while it is participating in the
dynamics) is nice and small, well below the critical
shift.



In [1]:
(T.lamb_shift(transmon, readout, cc_ro) / (2 * np.pi)).to(ureg.megahertz)

-4.745353647488288 megahertz

It is important to keep $C_c\ll C_Σ,C_B$ and $g\ll \abs{Δ}\lesssim ω_q +
ω_B$. Otherwise, everything variale.
Also, the anharmonicity is much smaller than $Δ$. Basically
everything in the following should be small ($\sim .01$).



In [1]:
print(T.dispersive_metrics(readout_transmon, readout, cc_ro))

{'Anharmonicity, α/Δ': 0.24122807017543857, 'Coupling Strength, g/Δ': 0.0823322893527073, 'RWA counter-rotating': 0.09836065573770493, 'Coupling/Transmon Cap': 0.05977702780049465, 'Coupling/CPW Cap': 0.00268}

More input is needed here:

-   quality factor of cavity after coupling to transmission line&#x2026;
-   this will show us, if the disperive shift is large enough



## Drive



The readout's restriction is the anharmonicity, although this
constraint is weak, as we can use DRAG (Blais, Alexandre and Grimsmo, Arne L. and Girvin, S. M. and Wallraff, Andreas, 2021).

The question is: what is a typical pulse amplitude?
I've found numbers in (Bardin, Joseph, ????) that say $\SIrange{50}{500}{\micro\volt}$
are normal.

This is the (rough) limit on the coupling to the drive.



In [1]:
formulas.C_d_max(T.Transmon.E_C, T.Transmon.E_J, T.V_Dmax)

0.2239109915991125 femtofarad

This depends on how hard we can attenuate the drive. Lower is better,
generally, as we don't want the qubit to relax into the drive line.

This is much smaller than the coupling to the bus and corresponds to
drive times that are much greater than



In [1]:
(2 * np.pi / T.Transmon.α).to(ureg.microsecond)

0.01 microsecond

which is still $1/10$ of $T_{1/2}$. This could get tight, but
thanks to DRAG this shouldn't be a big problem. Maybe we can also
detune the Qubit before flipping it.

